In [3]:
import pandas as pd
import gensim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy
%run calculateSentence.ipynb
import preprocessing
import tfidf
pd.set_option('max_colwidth', 5000)

In [4]:
#Importazione DB

data = pd.read_csv('DB/Preprocessed_data.csv')

data = data[data['processed_title'].notna()]

#data.head(10)

In [ ]:
#Importazione WordEmbeddings

word_2_vec_model = gensim.models.word2vec.Word2Vec.load('DB/word2vec_trained.bin')
word_2_vec_model

In [ ]:
def normalize(words):
    words = preprocessing.clear_text(words,"w2v")
    return words

In [ ]:
def deleteRow(data):
    
    row = []
                
    for tmp in data:
        if tmp[2] != []:
            row.append(tmp)
    
    return row

In [ ]:
def w2v_result(txt, tag):
    stringSearch = normalize(txt)

    #print(stringSearch)

    numberResult = "5"

    #Viene vettorizzata la query dell'utente

    vectorSearch = np.array([questionToVec(stringSearch, word_2_vec_model)])
    
    #Carico il file con gli embeddings dei titoli sui quali effettuare la corrispondenza
    allTitleEmb = pd.read_csv('DB/titleEmbeddings.csv').values

    #Calcolo della similarità del coseno for le query e tutti i titoli

    similarityCosine = pd.Series(cosine_similarity(vectorSearch, allTitleEmb)[0])
    
    #Personalizzazione della misura di similarità

    #similarityCosine = similarityCosine*(1+0.4*data.overall_scores_norm + 0.1*(data.sentiment_polarity))
    similarityCosine = similarityCosine*(1+0.4*data.score)
    
    #Lista che conterrà coppie i,j con i indice e j score di similarità
    results = []
    
    #Se tag è null vuol dire che siamo alla prima iterazione e non dobbiamo filtrare su nessun tag
    if tag == None :   
        #Una volta calcolati gli score di similarità rendo i 10 più alti
        for i,j in similarityCosine.nlargest(int(numberResult)).iteritems():
            #print(i, data.Body[i])
            #Filtro i tag da rendere come risultato, eliminando quelli che sono già presenti nella string search per facilitare prossimi passaggi 
            tags = data.iloc[[i]].tags.item().split('|')
            #print(tags)
            for t in tags:
                if t in stringSearch:
                    tags.remove(t)
                    
            results.append(tuple([data.iloc[[i]].id.values[0], j, tags]))
        results = deleteRow(results)
        
        
    #Entriamo nell'else se abbiamo il tag che deve essere incluso nei nostri risultati, 
    #e eliminiamo i record che non lo contengono dai result 
    else : 
        
        for i,j in similarityCosine.nlargest(int(numberResult)).iteritems():
            
            if tag in data.iloc[[i]].tags:
                results.append(tuple([data.iloc[[i]].id.values[0], j, data.iloc[[i]].tags]))
                
            
    #print(results)
    return results

In [ ]:
#w2v_result('problem loading csv file', None )

In [6]:
data.iloc[[9417]]

,id,title,tags,score,post_corpus,questions,processed_title
9418,17673314,Date Time convert from CSV to MySQL,mysql|datetime,-0.000505,date time convert csv mysql csv file loading mysql db one fields datatime ntp time format ex gmt wed may want convert acceptable format store mysql like csv file start loading mysql done loading csv file told done using python please help thanks advance convert ntp formatted dates using function loading csv file hope helps,"Date Time convert from CSV to MySQL I have a CSV file which i am loading into MySQL DB, one of the fields is DataTime in NTP time format ex: 10:14:18.531 gmt Wed May 15 2013 I want to convert it to an acceptable format to store it in MySQL like 2013-05-15 10:14:18 . Should i do this to the CSV file before start loading it to MySQL or it can be done while loading the CSV file I have been told that this can be done using Python please help. Thanks in advance.",date time convert csv mysql


In [4]:
'''
index = tfidf.prova(data['post_corpus'], ['problem loading csv file'])

print(index)
'''

[115354   9417  20921  86894]


In [ ]:

'''
for i,j in cosine_sim.nlargest(int(5)).iteritems():
    #print(i, data.Body[i])
    #Filtro i tag da rendere come risultato, eliminando quelli che sono già presenti nella string search per facilitare prossimi passaggi 
    titles = data.iloc[[i]].title
    print(title)
'''    

array([115354,   9417,  20921,  86894])

[[0.57735027]
 [0.        ]
 [0.        ]
 [0.        ]]
